# Serial read from arduino to graph

#### Instructions: 
1. Run first two cells below. 
2. Fill in widgets for required testing. 
3. Run execute script
4. Repeat step 2


#### User notes:  
- This script is written on Windows 10. Embrace yourself for several errors when using a different OS.
- There is a small delay in the code and therefore approximately 1.5 seconds is taken off the original chosen duration. (working on a fix)
- When executing multiple measurements in a row it sometimes occurs that ydata gets out of range (working on a fix). When this occurs, please reset the kernel & clear output.
- The CSV select bar searches for CSV files on your C: drive. No C: drive or files saved on other drive? Please adjust C: in first cell.
- When choosing unlimited duration on measurement. Depending on your OS and  pressing the following key(s) on your keyboard:
  - 2 times 'i' OR
  - cntrl+c   OR
  - delete

- The final plot contains four slider bars and a reset button:
 - [Alpha] Change Alpha value between 0.9 and 0.999 
 - [Beta]  Change Beta value between 0.9 and 0.999 
 - [Zoom]  Change ∆x on x - axis. From 5 seconds up to max timeframe.
 - [Pos]   Change x position on x-axis. From 0 to last second.
- Please set plot window to full size for correct use of sliderbars and buttons.





Please report any bugs to author:
victor@stoeten.eu


In [28]:
#import all needed libraries. In case one is missing > install via command by pasting line behind. [Windows only]
import serial                                     #pip install pyserial 
import os                                         #Standard library in Python
import csv                                        #Standard library in Python
from time import time                             #Standard library in Python
import datetime                                   #pip install DateTime   - standard library
import ipywidgets as widget                       #pip install ipywidgets - Big chance one will have this due to bouwplaats
from IPython.display import Javascript,display    #pip install ipython    - Big chance one will have this due to bouwplaats
from matplotlib import pyplot as plt              #pip install matplotlib - Big chance one will have this due to bouwplaats
from matplotlib.widgets import Slider, Button

#Remove if you want plots on seperate window. However this works better with the interactive plot at end of notebook
#%matplotlib inline                                

#Define functions
#Some functions for finding files in own directory and deleting files
def Csvfind(path_to_dir, suffix=".csv"):
    filenames = os.listdir(path_to_dir)           # save filenames in list [filenames]
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

#To find all .csv files on your pc saved at C: drive.
filenames = Csvfind("C:")  

# Delete a certain file by typing: Deletefile("filename.csv") 
def Deletefile(Delfilename):
    os.remove(Delfilename)

#function to run all cells below selected cell.
def Run_all(ev):
    
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

In [29]:
#Decimals on Y-data and time-data. Highly impacts size of savefiles!
decimalsy = 5
decimalstime = 5

#alignment settings of all widgets
align= dict(_css = (('.widget-label', 'min-width', '15ex'),),margin = '0px 0px 5px 22px')                          #Create better layout for widgets

#Create widgets
Limiteddur = widget.RadioButtons(options=['Yes', 'No'],description='Limited duration :',value='No', disabled=False, **align)  #Save file or not?
Tijd = widget.IntText(value='10', description='Duration [s]:',disabled=False, **align)                             #Choose duration of measurement
Save = widget.RadioButtons(options=['Yes', 'No'],description='Save file:', disabled=False, **align)                #Save file or not?
Testname = widget.Text(value='',placeholder='Type here',description='Save file as:',disabled=False, **align)       #Save file as:... +datestamp
Import = widget.RadioButtons(options=['Yes', 'No'],value='No',description='Import data:', disabled=False, **align) #Import data? -> no measurement is done
Importname = widget.Dropdown(options=filenames,description='File name:',disabled=False, **align)                   #Choose file you want to import

#Display widgets
display(Limiteddur,Tijd,Save,Testname,Import,Importname)


#Run button
button = widget.Button(description="Execute script", button_style='danger', **dict(_css = (('.widget-label', 'min-width', '15ex'),),margin = '50px 200px 20px 200px')) 
button.on_click(Run_all)                                                                                           #runs function run all
display(button)


<IPython.core.display.Javascript object>

#### Process all chosen values in widgets and write them to scriptsvariables below.

In [30]:
limdur = Limiteddur.value                      
savedata = Save.value
name = Testname.value
importdata = Import.value
namefile = str(Importname.value)

if limdur == str('No'):
    duration = 99999999  #creating an "unlimited" amount of time of three years.
else:
    duration = Tijd.value

#### Import data or take measurement

In [31]:
#import data when import date is chosen
if importdata == str('Yes'):
    savedata = str('No')
    ydata=[]
    timepoints=[]
    duration= []

    with open(namefile) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        duration = (float(next(csvReader)[2]))
        for row in csvReader:
            timepoints.append(float(row[0]))
            ydata.append(float(row[1]))

#Executes measurement
else:
    #Define the input of arduino:
    serial_port = 'Com10';
    baud_rate = 19200;
    ser = serial.Serial(serial_port, baud_rate)

    #Define variables:
    timepoints = [0]
    ydata = []
    run = True
    start_time = time()
    progress = widget.FloatProgress(min=0, max=duration*baud_rate/47)   #progressbar, only progresses with pre-determined duration
    display(progress)

    #Append values of Ydata and time to lists.
    try:
        while True:
            line = ser.readline().split();                              # Removes letters from serial input
            ydata.append(round(float(line[0])*5.5/1024,decimalsy))      # Creates floatingpoint of serial input and appends it to ydata list
            timepoints.append(round(time()-start_time,decimalstime))    # Creates floatingpoint timestamp and appends it to timepoints list
            current_time = timepoints[-1]                               # 
            progress.value += 1                                         # +1 tick on progressbar
            if timepoints[-1] > duration: break                         # KAN MOGELIJK WORDEN WEGGEHAALD!
    except KeyboardInterrupt:
        pass
    
    duration = timepoints[-1]                                           # Duration actual total duration incl. delay.
    
    ser.close()                                                         # Shut down serial input
    
    #Removes abnormal first value of created dataset. This sometimes happens and sometimes not.
    if timepoints[0] == 0: 
        ydata = ydata[1:]
        timepoints = timepoints[1:-1]

IndexError: list index out of range

#### Save the data

In [ ]:
if savedata == str('Yes'): 
    now = datetime.datetime.now()
    now2 = (now.strftime("%Y_%m_%d_%H_%M"))
    
    with open(str(now2)+ "_" +str(name)+".csv","w") as out_file:
        out_string = " "                                            # First line of file contains duration, therefor done seperate.
        out_string += str(timepoints[0])                            # 
        out_string += "," + str(ydata[0])
        out_string += "," +str(duration)
        out_string += "\n"
        out_file.write(out_string)
        for i in range(len(ydata)-1):                               # Rest of lines done with loop.
            out_string = " "
            out_string += str(timepoints[i+1])
            out_string += "," + str(ydata[i+1])
            out_string += "\n"
            out_file.write(out_string)
    namefile = str(now2)+ "_" +str(name)+".csv"


#### Calculate drops and create interactive plot to determine Alpha (and Beta) for auto regression.

In [ ]:
counter = 0
for i in range(len(ydata)):
    if ydata[i-1] > 0.5 and ydata[i] < ydata[i-1] > ydata[i-2]:                                   #Top when datapoint is larger than last and following datapoint. > 0.5 to remove noise tops. 
        counter += 1    

#correction of time. Removes the ~1.5 seconds at start
real_duration = duration - timepoints[0]        
        
# Set up figures
fig, ax = plt.subplots()

plt.subplots_adjust(bottom=0.25)
ax.axis([timepoints[1],timepoints[-1],-0.1,4])
plt.suptitle(('Name: {}  Drops counted:{}'.format((namefile[:-4]),counter)), fontsize='18', fontweight='bold')
plt.xlabel('Time [s]', fontsize='14', fontstyle='italic')
plt.ylabel('Potential [V]', fontsize='14', fontstyle='italic')
plt.ylim([-0.1,4])
    
# New time dataset, removes first datapoint since 
timepoints2= timepoints[1:]                                                                     #removes first value of timepoints, since autoregression can only start from i=1
timepoints3= timepoints2[1:]                                                                    #removes first value of timepoints2, since autoregression can only start from i=1

#Initial values for ALpha and Beta for auto regression steps.
Alpha1 = 0.9895
Beta1 = 0.9805

#First auto regression of ydata.
ynew = [0]
for i in range(len(ydata)-2):
    ynew.append(Alpha1*ynew[i] + (1-Alpha1)*ydata[i+1])
        
#Double auto regression, auto regresses over first auto regression 
#Not sure if this is correct, but surely gives a smoother graph than first autoregression
Double = [0]
for i in range(len(ynew)-2):
    Double.append(Beta1*Double[i] + (1-Beta1)*ynew[i+1])
    
Singlesecond = [0]
for i in range(len(ynew)-2):
    #Singlesecond.append(Beta1*Singlesecond[i] + ((Alpha1*ynew[i] + (1*ydata[i+1]-Alpha1*ydata[i+1]) - Beta1*Alpha1*ynew[i] - (Beta1*ydata[i+1]-Alpha1*Beta1*ydata[i+1]))))
    Singlesecond.append(Beta1*Singlesecond[i] + (1-Beta1)*Alpha1*ynew[i] + (1-Alpha1-Beta1+Alpha1*Beta1)*ydata[i+1])
    #Singlesecond.append(Beta1*Singlesecond[i] + (1-Beta1)*Alpha1*Singlesecond[i-1] + (1-Alpha1-Beta1+Alpha1*Beta1)*ydata[i+1])
    #Singlesecond.append((Beta1*Singlesecond[i] + ((1-Alpha1-Beta1+Alpha1*Beta1)*ydata[i+1]) 
    #                    + ((-Alpha1 -Alpha1**2 +Alpha1*Beta1 +Beta1*Alpha1**2)*ydata[i+1]) 
    #                    + ((-Alpha1*Singlesecond[i]+Alpha1*Beta1**2 *Singlesecond[i])/(1-Beta1)))
    #                    /(1-(Alpha1/(1-Beta1))+((Alpha1*Beta1)/(1-Beta1))))
#print(Singlesecond)
    
#Graph both lines in plot
plt.plot(timepoints, ydata,marker='.',markersize=2,linestyle='-',markerfacecolor='red',         #Plots ydata
        color='blue', alpha=0.4, label='Y-data')
Singlereg, = plt.plot(timepoints2, ynew, linestyle='-', label='Single reg',
                      markerfacecolor='green',color='green', alpha=0.8)                         # Plot first autoregression
Doublereg, = plt.plot(timepoints3, Double, linestyle='-',
                      markerfacecolor='black',color='red', label='Double reg')                                      # Plot second autoregression
Singlesecondreg, = plt.plot(timepoints3, Singlesecond, linestyle='-',
                      markerfacecolor='black',color='black', label='Total reg')
plt.axhline(y=0.5, color='r', linestyle='-',alpha = 0.2)                                        # Plots horizontal line to visualize the noise filter.

plt.legend() 
    
#Sliders build up
axpos = plt.axes([0.125, 0.05, 0.65, 0.025], axisbg='grey')
axBeta  = plt.axes([0.125, 0.13, 0.65, 0.025], axisbg='grey')
axAlpha = plt.axes([0.125, 0.17, 0.65, 0.025], axisbg='grey')
axzoom = plt.axes([0.125, 0.09, 0.65, 0.025], axisbg='grey')

    
#Sliders settings
szoom = Slider(axzoom, 'Zoom', 2, timepoints[-1],timepoints[-1])
spos = Slider(axpos, 'Pos', timepoints[0], timepoints[-1])
sAlpha = Slider(axAlpha, 'Alpha', 0.9, 0.999, Alpha1)
sBeta = Slider(axBeta, 'Beta', 0.9, 0.999, Beta1)

#Run update of values
def update(val):
    Beta = sBeta.val                                                                            # Update beta value
    Alpha = sAlpha.val                                                                          # Update Alpha value
    ynew = [0]
    for i in range(len(ydata)-2):                                                               # Update of 1st regression
        ynew.append(Alpha*ynew[i] + (1-Alpha)*ydata[i+1])
    Double = [0]
    for i in range(len(ynew)-2):                                                                # Update of 2nd regression
        Double.append(Beta*Double[i] + (1-Beta)*ynew[i+1])
    #Singlesecond = [0]
    #for i in range(len(ynew)-2):
    #    Singlesecond.append(Beta*Double[i]+(1-Beta)*(Alpha*ynew[i]+(1-Alpha)*ydata[i+1]))
    Singlereg.set_ydata(ynew)
    Doublereg.set_ydata(Double)
    Singlesecondreg.set_ydata(Singlesecond)
    zoom = szoom.val
    pos = spos.val
    ax.axis([pos,pos+zoom,-0.1,4])
    fig.canvas.draw_idle()
    
#Reset button
resetax = plt.axes([0.83, 0.05, 0.07, 0.03])
buttonall = Button(resetax, 'Reset all', color='grey', hovercolor='0.975')
buttonAB = Button(plt.axes([0.83, 0.13, 0.07, 0.03]), 'Reset alpha beta', color='grey', hovercolor='0.975')

def resetab(event):
    sBeta.reset()
    sAlpha.reset()

def resetall(event):
    sBeta.reset()
    sAlpha.reset()
    szoom.reset()
    spos.reset()
    ax.axis([timepoints[1],timepoints[-1],-0.1,4])

#Update when bars are used
spos.on_changed(update)
sAlpha.on_changed(update)
sBeta.on_changed(update)
szoom.on_changed(update)

#Reset when button pressed. This sometimes takes a while
buttonall.on_clicked(resetall)
buttonAB.on_clicked(resetab)
    
#Show plot
plt.show()

In [ ]:
Javascript('IPython.notebook.execute_cell_range(1,3)') #runs first two cells to update the list of csv files.


In [ ]:
#len(timepoints)/timepoints[-1]

In [ ]:
ser.close()
